# Cargamos los datos 

In [ ]:
df = pd.read_pickle('datos/datos_actualizados.pkl')
df.head(2)

# Creación del esquema para SQL 

## Conexión con el servidor de SQL y Creación BBDD

In [ ]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="admin" # aquí tendréis que poner vuestra contraseña de MySQL
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
mydb = crear_bbdd("clima")

## Creación de tablas

In [ ]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

# definimos tablas a insertar

In [ ]:
tabla_tiempo = '''
CREATE TABLE IF NOT EXISTS `clima`.`tiempo` (
  `idtiempo` INT NOT NULL AUTO_INCREMENT,
  `timepoint`INT NOT NULL,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idtiempo`))
ENGINE = InnoDB;
'''

In [ ]:
tabla_localidades = '''
CREATE TABLE IF NOT EXISTS `clima`.`localidades` (
  `idlocalidad` INT NOT NULL AUTO_INCREMENT,
  `latitud` DECIMAL(6,4) NOT NULL,
  `longitud` DECIMAL(6,4) NOT NULL,
  `ciudad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idlocalidad`))
ENGINE = InnoDB;
'''

In [ ]:
tabla_mediciones = '''
CREATE TABLE IF NOT EXISTS `clima`.`mediciones` (
  `idmedicion` INT NOT NULL AUTO_INCREMENT,
  `cloudcover` INT NOT NULL,
  `lifted_index` INT NOT NULL,
  `prec_type` VARCHAR(10) NOT NULL,
  `prec_amount` INT NOT NULL,
  `temp2m` INT NOT NULL,
  `rh2m` INT NOT NULL,
  `weather` VARCHAR(15) NOT NULL,
  `wind_direction` VARCHAR(15) NOT NULL,
  `wind_speed` INT NOT NULL,
  `seeing` DECIMAL NOT NULL,
  `transparency` INT NOT NULL,
  `idtiempo` INT NOT NULL,
  `idlocalidad` INT NOT NULL,
  PRIMARY KEY ( `idmedicion`, `idtiempo`, `idlocalidad`),
  INDEX `fk_mediciones_tiempo1_idx` (`idtiempo` ASC) ,
  INDEX `fk_mediciones_localidades1_idx` (`idlocalidad` ASC) ,
  CONSTRAINT `fk_mediciones_tiempo1`
    FOREIGN KEY (`idtiempo`)
    REFERENCES `clima`.`tiempo` (`idtiempo`),
  CONSTRAINT `fk_mediciones_localidades1`
    FOREIGN KEY (`idlocalidad`)
    REFERENCES `clima`.`localidades` (`idlocalidad`))
ENGINE = InnoDB;

'''

Llamamos a la función para crear cada una de las tablas de nuestra BBDD.

In [ ]:
crear_insertar_tabla("clima","admin", tabla_tiempo)

In [ ]:
crear_insertar_tabla("clima", "admin", tabla_localidades)

In [ ]:
crear_insertar_tabla("clima", "admin", tabla_mediciones)

## Insercción de datos 

In [ ]:
# manos a la obra 

for indice, fila in df.iterrows(): # itreamos por el dataframe.
    
    # definimos nuestra query, igual que si lo hicieramos en workbench. ⚠️ Como estamos definiendo nuestra query en varias líneas usamos las triples comillas
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_tiempo = f"""
            INSERT INTO tiempo (timepoint, fecha) # como la primary key la hemos puesto como autoincremental no hace falta introducirla en el insert
            VALUES ( "{fila["timepoint"]}", "{fila['fecha']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    crear_insertar_tabla("clima", "admin", query_tiempo)

# chequear si existe en mi tabla

In [ ]:
# con esta query extraermos todos los valores únicos que tenemos en nuestra BBDD para la ciudad.

def check_ciudades(contraseña, nombre_bbdd):
    
    cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}")
    mycursor = cnx.cursor()


    query_existe_ciudad = f"""
            SELECT DISTINCT ciudad FROM localidades
            """
    mycursor.execute(query_existe_ciudad)
    ciudades = mycursor.fetchall()
    return ciudades

In [ ]:
# qué nos devuelve esta función 

check_ciudades("admin", "clima")

In [ ]:
# Tenemos una lista con una tupla, por lo que nos tenemos que quedarnos con el elemento 0. 

check_ciudades("admin", "clima")[0]

Sobre este resultado ya podremos chequear si la ciudad que queremos insertar esta en nuestra BBDD o no. Vamos manos a la obra. 

In [ ]:
for indice, fila in df.iterrows():
    
    # creamos la query que no inserta datos en la tabla localidades
    query_localidad = f"""
                INSERT INTO localidades (latitud, longitud, ciudad) 
                VALUES ( "{fila["latitud"]}", "{fila['longitud']}", "{fila['ciudad']}");
                """
    # llamamos a la funcion check_ciudades para saber que ciudades tenemos en la BBDD
    ciudades = check_ciudades("admin", "clima")
    
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad. 
    # 
    if len(ciudades) == 0 or fila['ciudad'] not in ciudades[0]: 
        crear_insertar_tabla("clima","admin", query_localidad)

    else:
        print(f"{fila['ciudad']} ya esta en nuestra BBDD")    

## Insercción de datos en la tabla de mediciones:

In [ ]:
def sacar_id_ciudad(contraseña, nombre_bbdd, ciudad):
    
    # creamos la conexion con el servidor y la BBDD
    cnx = mysql.connector.connect(user='root', password= f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}")
    mycursor = cnx.cursor()

    # creamos la query para sacar el id de la ciudad que queramos
    query_sacar_id = f"SELECT idlocalidad FROM localidades WHERE ciudad = '{ciudad}'"
    
    # puede que el id de la ciudad que estemos intentando insertar no este en nuestra BBDD. Por lo que vamos a hacer un try except para que no nos falle el código
    try: 
        mycursor.execute(query_sacar_id)
        id_ = mycursor.fetchall()[0][0]
        return id_
    
    except: 
        return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "

In [ ]:
sacar_id_ciudad("admin", "clima", 'Madrid')

In [ ]:
def sacar_id_fecha(contraseña, nombre_bbdd, timepoint, fecha):
        
        cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                      host='127.0.0.1', database=f"{nombre_bbdd}")
        mycursor = cnx.cursor()


        query_sacar_id = f"SELECT idtiempo FROM tiempo WHERE timepoint = {timepoint} and fecha = '{fecha}'"
        
        try: 
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [ ]:
sacar_id_fecha("admin", "clima", 3, '2022-03-16' )

Una vez que somos capaces de sacar los `id` para la fecha y las localidades ya podremos insertar los datos en la BBDD. 

In [ ]:
for indice, fila in df.iterrows():
    
    id_ciudad = sacar_id_ciudad("admin", "clima", fila['ciudad'])
    id_fecha = sacar_id_fecha("admin", "clima", fila["timepoint"], fila["fecha"])
    
    query_medidas = f"""
                INSERT INTO mediciones (cloudcover, lifted_index, prec_type, prec_amount, temp2m, rh2m, weather, wind_direction, wind_speed, seeing, transparency, idtiempo, idlocalidad) 
                VALUES ({fila["cloudcover"]}, {fila['lifted_index']}, "{fila['prec_type']}", {fila['prec_amount']},{fila['temp2m']},{fila['rh2m']}, "{fila['weather']}","{fila['wind10m.direction']}",{fila['wind10m.speed']}, {fila['seeing']}, {fila['transparency']}, {id_fecha}, {id_ciudad});
                """
    crear_insertar_tabla("clima","admin", query_medidas)